In [2]:
from langchain_mistralai import ChatMistralAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

model = ChatMistralAI(endpoint="http://localhost:5050/v1/", api_key="not-needed")


file_path = "prova.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(splits, embedding_function)

docs = vectorstore.as_retriever(search_type="mmr")


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "input({input})"),
    ]
)


question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(docs, question_answer_chain)

results = rag_chain.invoke({"input": input("inserisci la richiesta")})

results



9


C:\Users\rizzel42\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'input': 'Describe abstract of this document prova.pdf',
 'context': [Document(page_content='promising amplified capabilities. However, ethical considera-\ntions are paramount, emphasizing responsible development and\ndeployment to ensure positive integration into our lives.\nReferences\n[Bahng et al. , 2022 ]Hyojin Bahng, Ali Jahanian, Swami\nSankaranarayanan, and Phillip Isola. Exploring visual\nprompts for adapting large-scale models. arXiv preprint\narXiv:2203.17274 , 2022.\n[Brown et al. , 2020 ]Tom B. Brown, Benjamin Mann, Nick\nRyder, Melanie Subbiah, Jared Kaplan, Prafulla Dhariwal,\nArvind Neelakantan, Pranav Shyam, Girish Sastry, Amanda\nAskell, Sandhini Agarwal, Ariel Herbert-V oss, Gretchen\nKrueger, Tom Henighan, Rewon Child, Aditya Ramesh,\nDaniel M. Ziegler, Jeffrey Wu, Clemens Winter, Christo-\npher Hesse, Mark Chen, Eric Sigler, Mateusz Litwin, Scott\nGray, Benjamin Chess, Jack Clark, Christopher Berner,\nSam McCandlish, Alec Radford, Ilya Sutskever, and Dario\nAmodei